In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fon-audios-raw/train-ag-00009-of-00012.parquet
/kaggle/input/fon-audios-raw/train-ac-00005-of-00012.parquet
/kaggle/input/fon-audios-raw/train-aa-00006-of-00012.parquet
/kaggle/input/fon-audios-raw/train-p-00004-of-00010.parquet
/kaggle/input/fon-audios-raw/train-bg-00010-of-00012.parquet
/kaggle/input/fon-audios-raw/train-s-00003-of-00010.parquet
/kaggle/input/fon-audios-raw/train-bc-00005-of-00012.parquet
/kaggle/input/fon-audios-raw/train-ah-00010-of-00012.parquet
/kaggle/input/fon-audios-raw/train-ab-00002-of-00012.parquet
/kaggle/input/fon-audios-raw/train-aq-00009-of-00011.parquet
/kaggle/input/fon-audios-raw/train-b-00002-of-00003.parquet
/kaggle/input/fon-audios-raw/train-i-00005-of-00010.parquet
/kaggle/input/fon-audios-raw/train-ap-00006-of-00011.parquet
/kaggle/input/fon-audios-raw/train-n-00002-of-00010.parquet
/kaggle/input/fon-audios-raw/train-k-00004-of-00010.parquet
/kaggle/input/fon-audios-raw/train-z-00007-of-00012.parquet
/kaggle/input/fon-audios-raw/tr



# Academi Pionners 3 : Test Pipeline Transcription Fon

**@hopsyder - Kaggle Free Tier Optimized**


# ===============================================
# INSTALLATION ET CONFIGURATION INITIALE
# ===============================================

In [2]:
!pip install torch
!pip install openai-whisper
!pip install yt-dlp

print("🔧 Installation des outils de décompression...")
!pip install -q py7zr rarfile patool
!pip install -q librosa soundfile pydub tqdm

!pip install rarfile
!apt-get install unrar


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.10.19:
      Successfully uninstalled nvidia-curand-cu12-10.3.10.19
  Attempt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import librosa
import soundfile as sf
from pydub import AudioSegment
import whisper
import torch

import yt_dlp
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime, timedelta
import time
from tqdm.auto import tqdm

import os
import sys
import shutil
import zipfile
import tarfile
import rarfile
import py7zr
from pathlib import Path
import pandas as pd
import numpy as np
import json
from datetime import datetime
import warnings


# Configuration pour Kaggle
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# ===============================================
# CLASSE EXTRACTEUR DATASET
# ===============================================

In [4]:
class FonDatasetExtractor:
    """
    Extracteur et processeur de dataset audio Fon
    @hopsyder - Optimisé pour Kaggle
    """
    
    def __init__(self, input_path="/kaggle/input/fon-audios-raw", 
                 output_path="/kaggle/working/fon_dataset_extracted"):
        """
        Initialisation de l'extracteur
        
        Args:
            input_path: Chemin vers le dataset compressé
            output_path: Chemin de sortie pour les fichiers extraits
        """
        self.input_path = Path(input_path)
        self.output_path = Path(output_path)
        self.extracted_files = []
        self.audio_files = []
        self.metadata = {}
        
        # Création du dossier de sortie
        self.output_path.mkdir(parents=True, exist_ok=True)
        
        print(f"📁 Dossier input: {self.input_path}")
        print(f"📁 Dossier output: {self.output_path}")
        
        # Vérification de l'existence du dataset
        if not self.input_path.exists():
            raise FileNotFoundError(f"Dataset non trouvé: {self.input_path}")
        
        print("✅ Extracteur initialisé!")
    
    def detect_archive_type(self, file_path):
        """Détection automatique du type d'archive"""
        file_path = Path(file_path)
        extension = file_path.suffix.lower()
        
        archive_types = {
            '.zip': 'zip',
            '.tar': 'tar',
            '.tar.gz': 'tar.gz',
            '.tgz': 'tar.gz',
            '.tar.bz2': 'tar.bz2',
            '.tar.xz': 'tar.xz',
            '.rar': 'rar',
            '.7z': '7z',
            '.gz': 'gz'
        }
        
        # Vérification extension composée
        if file_path.name.endswith('.tar.gz'):
            return 'tar.gz'
        elif file_path.name.endswith('.tar.bz2'):
            return 'tar.bz2'
        elif file_path.name.endswith('.tar.xz'):
            return 'tar.xz'
        
        return archive_types.get(extension, 'unknown')
    
    def extract_archive(self, archive_path, extract_to=None):
        """
        Extraction d'archive avec gestion de tous les formats
        
        Args:
            archive_path: Chemin vers l'archive
            extract_to: Dossier de destination (optionnel)
        """
        archive_path = Path(archive_path)
        extract_to = extract_to or self.output_path
        extract_to = Path(extract_to)
        
        archive_type = self.detect_archive_type(archive_path)
        
        print(f"📦 Extraction de {archive_path.name} (type: {archive_type})")
        
        try:
            if archive_type == 'zip':
                with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                    extracted_files = zip_ref.namelist()
            
            elif archive_type in ['tar', 'tar.gz', 'tar.bz2', 'tar.xz']:
                mode_map = {
                    'tar': 'r',
                    'tar.gz': 'r:gz',
                    'tar.bz2': 'r:bz2',
                    'tar.xz': 'r:xz'
                }
                
                with tarfile.open(archive_path, mode_map[archive_type]) as tar_ref:
                    tar_ref.extractall(extract_to)
                    extracted_files = tar_ref.getnames()
            
            elif archive_type == 'rar':
                with rarfile.RarFile(archive_path) as rar_ref:
                    rar_ref.extractall(extract_to)
                    extracted_files = rar_ref.namelist()
            
            elif archive_type == '7z':
                with py7zr.SevenZipFile(archive_path, mode='r') as z7_ref:
                    z7_ref.extractall(extract_to)
                    extracted_files = z7_ref.getnames()
            
            elif archive_type == 'gz':
                # Pour les fichiers .gz simples
                import gzip
                output_file = extract_to / archive_path.stem
                with gzip.open(archive_path, 'rb') as gz_file:
                    with open(output_file, 'wb') as out_file:
                        shutil.copyfileobj(gz_file, out_file)
                extracted_files = [output_file.name]
            
            else:
                raise ValueError(f"Type d'archive non supporté: {archive_type}")
            
            print(f"✅ Extraction réussie: {len(extracted_files)} fichiers")
            return extracted_files
            
        except Exception as e:
            print(f"❌ Erreur extraction: {e}")
            return []
    
    def scan_input_directory(self):
        """Scan du dossier input pour identifier les archives"""
        print("🔍 Scan du dossier input...")
        
        archives_found = []
        
        for item in self.input_path.rglob("*"):
            if item.is_file():
                archive_type = self.detect_archive_type(item)
                if archive_type != 'unknown':
                    archives_found.append({
                        'path': item,
                        'name': item.name,
                        'size': item.stat().st_size,
                        'type': archive_type
                    })
        
        if not archives_found:
            print("⚠️ Aucune archive trouvée, recherche de fichiers audio directs...")
            # Recherche de fichiers audio non compressés
            audio_extensions = ['.wav', '.mp3', '.m4a', '.flac', '.ogg', '.aac']
            for ext in audio_extensions:
                audio_files = list(self.input_path.rglob(f"*{ext}"))
                if audio_files:
                    print(f"📄 Trouvé {len(audio_files)} fichiers {ext}")
                    self.audio_files.extend(audio_files)
        
        print(f"📦 Archives trouvées: {len(archives_found)}")
        for archive in archives_found:
            size_mb = archive['size'] / (1024 * 1024)
            print(f"   - {archive['name']} ({archive['type']}) - {size_mb:.1f} MB")
        
        return archives_found
    
    def extract_all_archives(self):
        """Extraction de toutes les archives trouvées"""
        archives = self.scan_input_directory()
        
        if not archives:
            print("ℹ️ Aucune archive à extraire")
            return
        
        total_extracted = 0
        
        for archive in archives:
            print(f"\n📦 Traitement: {archive['name']}")
            
            # Création d'un sous-dossier pour chaque archive
            archive_output_dir = self.output_path / archive['path'].stem
            archive_output_dir.mkdir(exist_ok=True)
            
            # Extraction
            extracted_files = self.extract_archive(archive['path'], archive_output_dir)
            
            if extracted_files:
                self.extracted_files.extend(extracted_files)
                total_extracted += len(extracted_files)
                
                # Recherche des fichiers audio dans les extraits
                self.find_audio_files_in_directory(archive_output_dir)
        
        print(f"\n✅ Extraction terminée: {total_extracted} fichiers extraits")
        print(f"🎵 Fichiers audio trouvés: {len(self.audio_files)}")
    
    def find_audio_files_in_directory(self, directory):
        """Recherche récursive des fichiers audio"""
        audio_extensions = ['.wav', '.mp3', '.m4a', '.flac', '.ogg', '.aac', '.wma']
        
        for ext in audio_extensions:
            audio_files = list(Path(directory).rglob(f"*{ext}"))
            self.audio_files.extend(audio_files)
    
    def analyze_audio_files(self):
        """Analyse des fichiers audio extraits"""
        if not self.audio_files:
            print("⚠️ Aucun fichier audio à analyser")
            return None
        
        print(f"🔍 Analyse de {len(self.audio_files)} fichiers audio...")
        
        audio_metadata = []
        
        for audio_file in tqdm(self.audio_files, desc="Analyse audio"):
            try:
                # Informations de base
                file_info = {
                    'filename': audio_file.name,
                    'filepath': str(audio_file),
                    'size_mb': audio_file.stat().st_size / (1024 * 1024),
                    'extension': audio_file.suffix.lower()
                }
                
                # Analyse audio avec librosa (échantillonnage pour rapidité)
                try:
                    # Chargement de seulement 30 secondes pour l'analyse
                    y, sr = librosa.load(str(audio_file), duration=30, sr=None)
                    
                    file_info.update({
                        'duration_analyzed': len(y) / sr,
                        'sample_rate': sr,
                        'channels': 1 if y.ndim == 1 else y.shape[0],
                        'rms_energy': float(np.sqrt(np.mean(y**2))),
                        'zero_crossing_rate': float(np.mean(librosa.feature.zero_crossing_rate(y)[0])),
                        'spectral_centroid': float(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)[0]))
                    })
                    
                    # Estimation de la durée totale via pydub (plus rapide)
                    audio_segment = AudioSegment.from_file(str(audio_file))
                    file_info['total_duration'] = len(audio_segment) / 1000.0  # en secondes
                    
                except Exception as e:
                    file_info.update({
                        'error': str(e),
                        'analysis_status': 'failed'
                    })
                
                audio_metadata.append(file_info)
                
            except Exception as e:
                print(f"❌ Erreur analyse {audio_file.name}: {e}")
                continue
        
        # Création du DataFrame
        metadata_df = pd.DataFrame(audio_metadata)
        
        # Statistiques
        if not metadata_df.empty:
            print(f"\n📊 Statistiques du dataset:")
            print(f"   - Nombre de fichiers: {len(metadata_df)}")
            print(f"   - Taille totale: {metadata_df['size_mb'].sum():.1f} MB")
            print(f"   - Durée totale: {metadata_df['total_duration'].sum()/3600:.1f} heures")
            print(f"   - Formats: {metadata_df['extension'].value_counts().to_dict()}")
            print(f"   - Sample rates: {metadata_df['sample_rate'].value_counts().to_dict()}")
        
        return metadata_df
    
    def create_dataset_structure(self):
        """Création d'une structure de dataset organisée"""
        print("🏗️ Création de la structure de dataset...")
        
        # Dossiers organisés
        structured_path = self.output_path / "structured_dataset"
        
        folders = {
            'raw_audio': structured_path / "raw_audio",
            'processed_audio': structured_path / "processed_audio", 
            'metadata': structured_path / "metadata",
            'transcriptions': structured_path / "transcriptions",
            'analysis': structured_path / "analysis"
        }
        
        # Création des dossiers
        for folder_name, folder_path in folders.items():
            folder_path.mkdir(parents=True, exist_ok=True)
            print(f"   📁 {folder_name}: {folder_path}")
        
        # Copie des fichiers audio vers raw_audio
        if self.audio_files:
            print(f"📋 Copie de {len(self.audio_files)} fichiers audio...")
            
            for audio_file in tqdm(self.audio_files[:50], desc="Copie fichiers"):  # Limite pour Kaggle
                try:
                    dest_file = folders['raw_audio'] / audio_file.name
                    shutil.copy2(audio_file, dest_file)
                except Exception as e:
                    print(f"⚠️ Erreur copie {audio_file.name}: {e}")
        
        self.structured_path = structured_path
        return folders
    
    def save_metadata_and_analysis(self, metadata_df):
        """Sauvegarde des métadonnées et analyses"""
        if metadata_df is None or metadata_df.empty:
            print("⚠️ Pas de métadonnées à sauvegarder")
            return
        
        # Sauvegarde CSV
        metadata_file = self.output_path / "fon_dataset_metadata.csv"
        metadata_df.to_csv(metadata_file, index=False)
        print(f"💾 Métadonnées sauvées: {metadata_file}")
        
        # Sauvegarde JSON détaillée
        json_file = self.output_path / "fon_dataset_analysis.json"
        analysis_data = {
            'extraction_info': {
                'timestamp': datetime.now().isoformat(),
                'total_files': len(self.audio_files),
                'total_size_mb': metadata_df['size_mb'].sum(),
                'total_duration_hours': metadata_df['total_duration'].sum() / 3600
            },
            'file_statistics': {
                'formats': metadata_df['extension'].value_counts().to_dict(),
                'sample_rates': metadata_df['sample_rate'].value_counts().to_dict(),
                'avg_duration': metadata_df['total_duration'].mean(),
                'avg_size_mb': metadata_df['size_mb'].mean()
            },
            'quality_metrics': {
                'avg_rms_energy': metadata_df['rms_energy'].mean(),
                'avg_spectral_centroid': metadata_df['spectral_centroid'].mean(),
                'files_with_errors': len(metadata_df[metadata_df['analysis_status'] == 'failed'])
            }
        }
        
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(analysis_data, f, indent=2, ensure_ascii=False)
        
        print(f"💾 Analyse JSON sauvée: {json_file}")
        
        # Visualisations
        self.create_visualizations(metadata_df)
    
    def create_visualizations(self, metadata_df):
        """Création de visualisations du dataset"""
        print("📊 Génération des visualisations...")
        
        # Configuration
        plt.style.use('seaborn-v0_8')
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Fon Dataset Analysis - @hopsyder', fontsize=16, fontweight='bold')
        
        # 1. Distribution des durées
        axes[0, 0].hist(metadata_df['total_duration'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Distribution des Durées')
        axes[0, 0].set_xlabel('Durée (secondes)')
        axes[0, 0].set_ylabel('Nombre de fichiers')
        
        # 2. Distribution des tailles
        axes[0, 1].hist(metadata_df['size_mb'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
        axes[0, 1].set_title('Distribution des Tailles')
        axes[0, 1].set_xlabel('Taille (MB)')
        axes[0, 1].set_ylabel('Nombre de fichiers')
        
        # 3. Formats de fichiers
        format_counts = metadata_df['extension'].value_counts()
        axes[1, 0].pie(format_counts.values, labels=format_counts.index, autopct='%1.1f%%', startangle=90)
        axes[1, 0].set_title('Répartition des Formats')
        
        # 4. Sample rates
        sr_counts = metadata_df['sample_rate'].value_counts()
        axes[1, 1].bar(sr_counts.index.astype(str), sr_counts.values, color='orange', alpha=0.7)
        axes[1, 1].set_title('Sample Rates')
        axes[1, 1].set_xlabel('Sample Rate (Hz)')
        axes[1, 1].set_ylabel('Nombre de fichiers')
        axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        
        # Sauvegarde
        viz_file = self.output_path / "fon_dataset_visualization.png"
        plt.savefig(viz_file, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"📊 Visualisation sauvée: {viz_file}")


In [5]:
# ===============================================
# FONCTION PRINCIPALE D'EXTRACTION
# ===============================================

def extract_fon_dataset():
    """
    Fonction principale d'extraction et traitement du dataset Fon
    @hopsyder
    """
    print("🎬 === EXTRACTION DATASET FON AUDIO ===")
    print(f"📅 Début: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    try:
        # Initialisation de l'extracteur
        extractor = FonDatasetExtractor()
        
        # Extraction des archives
        print("\n📦 PHASE 1: EXTRACTION DES ARCHIVES")
        extractor.extract_all_archives()
        
        # Si pas d'archives, recherche directe des fichiers audio
        if not extractor.audio_files:
            print("\n🔍 PHASE 1b: RECHERCHE DIRECTE FICHIERS AUDIO")
            extractor.find_audio_files_in_directory(extractor.input_path)
        
        # Analyse des fichiers audio
        print("\n🎵 PHASE 2: ANALYSE DES FICHIERS AUDIO")
        metadata_df = extractor.analyze_audio_files()
        
        # Création de la structure organisée
        print("\n🏗️ PHASE 3: STRUCTURATION DU DATASET")
        folders = extractor.create_dataset_structure()
        
        # Sauvegarde des métadonnées
        print("\n💾 PHASE 4: SAUVEGARDE MÉTADONNÉES")
        extractor.save_metadata_and_analysis(metadata_df)
        
        # Résumé final
        print("\n🎉 === EXTRACTION TERMINÉE ===")
        if metadata_df is not None and not metadata_df.empty:
            print(f"📊 RÉSUMÉ:")
            print(f"   - Fichiers extraits: {len(extractor.audio_files)}")
            print(f"   - Taille totale: {metadata_df['size_mb'].sum():.1f} MB")
            print(f"   - Durée totale: {metadata_df['total_duration'].sum()/3600:.2f} heures")
            print(f"   - Formats: {metadata_df['extension'].value_counts().to_dict()}")
            
            # Fichiers de sortie
            print(f"\n📁 FICHIERS GÉNÉRÉS:")
            print(f"   - Dataset structuré: /kaggle/working/fon_dataset_extracted/structured_dataset/")
            print(f"   - Métadonnées: fon_dataset_metadata.csv")
            print(f"   - Analyse: fon_dataset_analysis.json")
            print(f"   - Visualisation: fon_dataset_visualization.png")
            
            return metadata_df, extractor
        else:
            print("⚠️ Aucun fichier audio traité")
            return None, extractor
            
    except Exception as e:
        print(f"❌ ERREUR CRITIQUE: {e}")
        import traceback
        traceback.print_exc()
        return None, None
    
    finally:
        print(f"\n⏱️ Fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [6]:
# ===============================================
# FONCTION UTILITAIRE POUR KAGGLE DATASET
# ===============================================

def create_kaggle_dataset_version():
    """
    Crée une nouvelle version du dataset pour votre profil Kaggle
    """
    print("📤 Préparation pour upload Kaggle Dataset...")
    
    # Vérification des fichiers requis
    required_files = [
        "/kaggle/working/fon_dataset_extracted/structured_dataset/",
        "/kaggle/working/fon_dataset_metadata.csv",
        "/kaggle/working/fon_dataset_analysis.json"
    ]
    
    existing_files = [f for f in required_files if os.path.exists(f)]
    
    print(f"✅ Fichiers prêts: {len(existing_files)}/{len(required_files)}")
    
    # Création du dataset-metadata.json pour Kaggle
    kaggle_metadata = {
        "title": "Fon Language Audio Dataset - Processed",
        "id": "votre-username/fon-audio-processed",
        "description": "Dataset audio en langue Fon extrait et traité avec métadonnées complètes",
        "tags": ["audio", "nlp", "african-languages", "fon", "speech-recognition"],
        "licenses": [{"name": "CC0-1.0"}],
        "collaborators": [],
        "data": []
    }
    
    with open("/kaggle/working/dataset-metadata.json", "w") as f:
        json.dump(kaggle_metadata, f, indent=2)
    
    print("📋 Métadonnées Kaggle créées: dataset-metadata.json")
    print("\n🚀 PROCHAINES ÉTAPES:")
    print("1. Téléchargez tous les fichiers de /kaggle/working/")
    print("2. Créez un nouveau dataset sur kaggle.com/datasets")
    print("3. Uploadez les fichiers avec dataset-metadata.json")
    print("4. Publiez votre dataset traité !")

# ===============================================
# EXÉCUTION PRINCIPALE
# ===============================================

if __name__ == "__main__":
    # Exécution de l'extraction
    metadata_df, extractor = extract_fon_dataset()
    
    # Préparation pour Kaggle Dataset
    if metadata_df is not None:
        create_kaggle_dataset_version()
    
    print("\n🔚 === SCRIPT TERMINÉ ===")

🎬 === EXTRACTION DATASET FON AUDIO ===
📅 Début: 2025-07-08 22:19:23
📁 Dossier input: /kaggle/input/fon-audios-raw
📁 Dossier output: /kaggle/working/fon_dataset_extracted
✅ Extracteur initialisé!

📦 PHASE 1: EXTRACTION DES ARCHIVES
🔍 Scan du dossier input...
⚠️ Aucune archive trouvée, recherche de fichiers audio directs...
📦 Archives trouvées: 0
ℹ️ Aucune archive à extraire

🔍 PHASE 1b: RECHERCHE DIRECTE FICHIERS AUDIO

🎵 PHASE 2: ANALYSE DES FICHIERS AUDIO
⚠️ Aucun fichier audio à analyser

🏗️ PHASE 3: STRUCTURATION DU DATASET
🏗️ Création de la structure de dataset...
   📁 raw_audio: /kaggle/working/fon_dataset_extracted/structured_dataset/raw_audio
   📁 processed_audio: /kaggle/working/fon_dataset_extracted/structured_dataset/processed_audio
   📁 metadata: /kaggle/working/fon_dataset_extracted/structured_dataset/metadata
   📁 transcriptions: /kaggle/working/fon_dataset_extracted/structured_dataset/transcriptions
   📁 analysis: /kaggle/working/fon_dataset_extracted/structured_dataset/a

# ===============================================
# CLASSE PRINCIPALE - PIPELINE TRANSCRIPTION FON
# ===============================================